<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/Marvel_series/Exploratory%20graph%20analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* Updated to GDS 2.0 version
* Link to original blog post: https://towardsdatascience.com/exploratory-network-analysis-of-marvel-universe-c557f4959048

In [1]:
!pip install neo4j

     |████████████████████████████████| 89 kB 4.0 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.2-py3-none-any.whl size=115365 sha256=37440807321d6c4c73bd85c38c1683cf854f8a8e68d3ed332d5782abd497de89
  Stored in directory: /root/.cache/pip/wheels/10/d6/28/95029d7f69690dbc3b93e4933197357987de34fbd44b50a0e4
Successfully built neo4j


I recommend you setup a [blank project on Neo4j Sandbox environment](https://sandbox.neo4j.com/?usecase=blank-sandbox), but you can also use other environment versions



In [2]:
# Define Neo4j connections
import pandas as pd
from neo4j import GraphDatabase

host = 'bolt://3.235.2.228:7687'
user = 'neo4j'
password = 'seats-drunks-carbon'
driver = GraphDatabase.driver(host,auth=(user, password))

In [3]:
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [4]:
import_queries = """

CALL apoc.schema.assert({Character:['name']},{Comic:['id'], Character:['id'], Event:['id'], Group:['id']});

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/Marvel/heroes.csv" as row
CREATE (c:Character)
SET c += row;

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/Marvel/groups.csv" as row
CREATE (c:Group)
SET c += row;

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/Marvel/events.csv" as row
CREATE (c:Event)
SET c += row;

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/Marvel/comics.csv" as row
CREATE (c:Comic)
SET c += apoc.map.clean(row,[],["null"]);

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/Marvel/heroToComics.csv" as row
MATCH (c:Character{id:row.hero})
MATCH (co:Comic{id:row.comic})
MERGE (c)-[:APPEARED_IN]->(co);

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/Marvel/heroToEvent.csv" as row
MATCH (c:Character{id:row.hero})
MATCH (e:Event{id:row.event})
MERGE (c)-[:PART_OF_EVENT]->(e);

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/Marvel/heroToGroup.csv" as row
MATCH (c:Character{id:row.hero})
MATCH (g:Group{id:row.group})
MERGE (c)-[:PART_OF_GROUP]->(g);

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/Marvel/heroToHero.csv" as row
MATCH (s:Character{id:row.source})
MATCH (t:Character{id:row.target})
CALL apoc.create.relationship(s,row.type, {}, t) YIELD rel
RETURN distinct 'done';

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/Marvel/heroStats.csv" as row
MATCH (s:Character{id:row.hero})
CREATE (s)-[:HAS_STATS]->(stats:Stats)
SET stats += apoc.map.clean(row,['hero'],[]);

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/Marvel/heroFlight.csv" as row
MATCH (s:Character{id:row.hero})
SET s.flight = row.flight;

MATCH (s:Stats)
WITH keys(s) as keys LIMIT 1
MATCH (s:Stats)
UNWIND keys as key
CALL apoc.create.setProperty(s, key, toInteger(s[key]))
YIELD node
RETURN distinct 'done';
"""

## Graph import

In [5]:
with driver.session() as session:
    for statement in import_queries.split(';'):
        try:
            session.run(statement.strip())
        except:
            pass

## Graph schema
In the center of the graph, there are characters, also known as heroes. They can appear in multiple comics, are part of an event, and can belong to a group. For some of the characters, we also know their stats like speed and fighting skills. Finally, we have social ties between characters that represent relative, ally, or enemy relationships.

There are 1105 characters that have appeared in 38875 comics.
We have stats for 470 of the characters. There are also 92 groups and 74 events stored in the graph.
## Exploratory graph analysis
To get to know our graph, we will begin with a basic graph data exploration process. First, we will take a look at the characters that have most frequently appeared in comics.

In [6]:
run_query("""
MATCH (c:Character)
RETURN c.name as character, 
       size((c)-[:APPEARED_IN]->()) as comics
ORDER BY comics DESC
LIMIT 5
""")

,character,comics
0,Spider-Man (1602),3357
1,Tony Stark,2354
2,Logan,2098
3,Steve Rogers,2019
4,Thor (Marvel: Avengers Alliance),1547


The top five most frequent characters come as no surprise. Spiderman is the most frequent or popular character. It is no wonder that they created a younger version of Spiderman just recently, given his popularity. Tony Stark, also known as Iron Man, is in second place. It seems that Logan, also known as Wolverine, was quite popular throughout history, but I think that his popularity slowly faded away in recent times. Steve Rogers, who goes by the more popular name Captain America, is also quite famous. It would seem that the recent Marvel movies showcased the more popular characters from the comics.

Next, we will look at how many comics were released throughout the decades. The year of the comic is stored as a string in our graph, so we can use the substringfunction to extract the decade.

In [7]:
run_query("""
MATCH (c:Comic)
RETURN substring(c.year, 0, 3) + "0" as decade, 
       count(*) as count
ORDER BY decade ASC
""")

,decade,count
0,1930,95
1,1940,584
2,1950,756
3,1960,4114
4,1970,1956
5,1980,2428
6,1990,3738
7,2000,8309
8,2010,11139
9,2020,19


Interesting to see that the first comics were produced in the 1930s. Some of the heroes are relatively senior by now. There was a spike in the 1960s and then gradual progression over the decades with 11.139 comics in the 2010s. The last column represents comics with a null date, so for around 6000 comics out of 38000, we don’t have the date available. And we haven’t scraped all the comics in the 2020s either.

Next, we will take a look at the most popular characters in the comics throughout the decades. We will iterate over comics and extract the top three most frequent heroes by the decade.

In [8]:
run_query("""
MATCH (c:Comic)<-[:APPEARED_IN]-(c1:Character)
WHERE NOT c.year = "null"
WITH substring(c.year,0,3) + "0" as decade, 
     c1.name as character, 
     count(*) as count
ORDER BY count DESC
RETURN decade, collect(character)[..3] as top_3_characters
ORDER BY decade
""")

,decade,top_3_characters
0,1930,"[Johnny Storm, Sub-Mariner, Archangel]"
1,1940,"[Johnny Storm, Two-Gun Kid, Steve Rogers]"
2,1950,"[Rawhide Kid, Tony Stark, Stephen Strange]"
3,1960,"[Thor (Marvel: Avengers Alliance), Spider-Man ..."
4,1970,"[Spider-Man (1602), Stephen Strange, Shang-Chi..."
5,1980,"[Logan, Tony Stark, Spider-Man (1602)]"
6,1990,"[Spider-Man (1602), Tony Stark, Steve Rogers]"
7,2000,"[Spider-Man (1602), Tony Stark, Logan]"
8,2010,"[Spider-Man (1602), Steve Rogers, Logan]"


It seems it all started with Johnny Storm, also known as the Human Torch. Iron Man (Tony Stark) was already popular in the 1950s, and Spiderman and Captain America (Steve Rogers) have risen in popularity in the 1960s. From then on, it seems that Spiderman, Wolverine, Iron Man, and Captain America win the popularity contest.
You might be wondering what the events are in our graph, so let’s take a look. We will examine the events with the highest count of participating heroes.

In [9]:
run_query("""
MATCH (e:Event)
RETURN e.title as event, 
       size((e)<-[:PART_OF_EVENT]-()) as count_of_heroes,
       e.start as start,
       e.end as end,
       e.description as description 
ORDER BY count_of_heroes DESC 
LIMIT 5
""")

,event,count_of_heroes,start,end,description
0,Fear Itself,132,2011-04-16 00:00:00,2011-10-18 00:00:00,"The Serpent, God of Fear and brother to the Al..."
1,Dark Reign,128,2008-12-01 00:00:00,2009-12-31 12:59:00,Norman Osborn came out the hero of Secret Inva...
2,Acts of Vengeance!,93,1989-12-10 00:00:00,2008-01-04 00:00:00,Loki sets about convincing the super-villains ...
3,Secret Invasion,89,2008-06-02 00:00:00,2009-01-25 00:00:00,The shape-shifting Skrulls have been infiltrat...
4,Civil War,86,2006-07-01 00:00:00,2007-01-29 00:00:00,After a horrific tragedy raises questions on w...


I have little to no idea what these events represent, but it is interesting to see that many characters participate. Most of the events span over less than a year, while the Acts of Vengeance spans over two decades. And judging by the description, Loki had something to do with it along with 92! other characters. Unfortunately, we don’t have the connection between comics and events stored in our graph to allow further analysis. If someone will scrape the Marvel API, I will gladly add it to the dataset.

Let’s also take a look at the biggest groups of characters.

In [10]:
run_query("""
MATCH (g:Group)
RETURN g.name as group, 
       size((g)<-[:PART_OF_GROUP]-()) as members
ORDER BY members DESC LIMIT 5
""")

,group,members
0,X-Men,41
1,Avengers,31
2,Defenders,26
3,Next Avengers,14
4,Guardians of the Galaxy,12


There are 41 characters in X-Men, which makes sense as they had a whole academy. You might be surprised by 31 members of Avengers, but in the comics, there were many members of Avengers, although most are former members.

Just because we can, let’s inspect if some members of the same group are also enemies.

In [11]:
run_query("""
MATCH (c1:Character)-[:PART_OF_GROUP]->(g:Group)<-[:PART_OF_GROUP]-(c2:Character)
WHERE (c1)-[:ENEMY]-(c2) and id(c1) < id(c2)
RETURN c1.name as character1, c2.name as character2, g.name as group
""")

,character1,character2,group
0,Logan,Sabretooth (House of M),X-Men
1,Logan,Mystique (House of M),X-Men
2,CAIN MARKO JUGGERNAUT,Logan,X-Men
3,CAIN MARKO JUGGERNAUT,Storm (Marvel Heroes),X-Men
4,Rogue (X-Men: Battle of the Atom),Warren Worthington III,X-Men


It seems that Logan does not get along with some of the other X-Men. For some of the characters, we also have the place of origin and education available, so let’s quickly look at that. During the scraping, I noticed a hero originated from Yugoslavia, so I wonder if there are more characters from Yugoslavia.

In [12]:
run_query("""
MATCH (c:Character)
WHERE c.place_of_origin contains "Yugoslavia"
RETURN c.name as character, 
       c.place_of_origin as place_of_origin,
       c.aliases as aliases
""")

,character,place_of_origin,aliases
0,Purple Man,"Rijeka, Yugoslavia","Killgrave the Purple Man, Killy"
1,Abomination (Ultimate),"Zagreb, Yugoslavia","Agent R-7, the Ravager of Worlds"


Two characters originated from today’s Croatia, which is less than two hours drive from where I live. Let’s also check out all the characters that completed their Ph.D. degree.

In [13]:
run_query("""
MATCH (c:Character)
WHERE c.education contains "Ph.D"
RETURN c.name as character, c.education as education
LIMIT 10
""")

,character,education
0,UNKNOWN ACHEBE,"Ph.D. in Law (Yale), degrees in Psychology, Po..."
1,PROFESSOR MENDEL STROMM MENDEL STROMM,Ph.D. in robotics
2,FRANKLIN HALL GRAVITON,Ph.D. in physics
3,Morbius,Ph.D in Biochemistry
4,Tony Stark,Ph.Ds in physics and electrical engineering
5,Hulk-dok,Ph.D in nuclear physics and two other fields
6,Sasquatch (Walter Langkowski),Ph.D. in physics from the Massachusetts Instit...
7,Professor X (Ultimate),"Ph.Ds in genetics, biophysics, psychology, and..."
8,Klaw,"Ph.D. in physics, bachelor’s degree in geology"
9,High Evolutionary,Uncompleted Ph.D at Oxford University


It looks like a lot of these heroes are quite employable. Only Nightshade seems a bit dodgy. It feels like something one would put on their LinkedIn profile to get noticed when searching for Ph.D. profiles. By the way, did you know that Professor X has four Ph.D.s and is also MD in psychiatry? Quite the educated men.
## Analyzing communities of allies and relatives
We have examined basic graph statistics, and now we will focus more on network analysis. We will investigate the social ties between characters.
To start, we will calculate the degree values for each relationship type between characters and display the heroes with the highest overall degree.

In [14]:
run_query("""
MATCH (c:Character)
RETURN c.name as name,
       size((c)-[:ALLY]->()) as allies,
       size((c)-[:ENEMY]->()) as enemies,
       size((c)-[:RELATIVE]->()) as relative
ORDER BY allies + enemies + relative DESC 
LIMIT 5
""")

,name,allies,enemies,relative
0,Scarlet Witch (Marvel Heroes),16,14,8
1,Thor (Marvel: Avengers Alliance),9,14,10
2,Invisible Woman (Marvel: Avengers Alliance),13,10,7
3,Logan,14,10,5
4,Karnak,6,2,17


Scarlet Witch and Thor seem to have the most direct enemies. Wolverine has the most allies but also many enemies. It looks like Triton has a big family with 17 direct relative relationships. We can use the `apoc.path.subgraphAll` procedure to examine the relatives' community of Triton.

In [15]:
run_query("""
MATCH p=(c:Character{name:"Triton"})
CALL apoc.path.subgraphAll(id(c), {relationshipFilter:"RELATIVE"})
YIELD nodes, relationships
RETURN nodes, relationships
""")

,nodes,relationships
0,"[(aliases, education, identity, name, id, plac...","[(), (), (), (), (), (), (), (), (), (), (), (..."


I never knew that some of the Marvel heroes have quite a big happy family. It wouldn’t be accurate if there weren’t a black sheep of the family present. Maximus looks like the family’s black sheep here as he has four enemies within the family. You might wonder why ally and enemy relationships are shown when we only traversed the relative ties. Neo4j Browser has a feature that displays all connections between nodes on the screen.

## Weakly Connected Components algorithm
The Weakly Connected Components is a part of almost every graph analysis workflow. It is used to find disconnected components or islands within the network. In this example, the graph consists of two components. Michael, Mark, and Doug belong to the first component, while Bridget, Alice, and Charles belong to the second component. We will apply the Weakly Connected Components algorithm to find the largest component of allied characters. As we don’t plan to run any other algorithms on this network, we will use the anonymous graph projection.

In [19]:
run_query("""
CALL gds.graph.project('allies', 'Character', 'ALLY')
""")

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'Character': {'label': 'Character', 'properti...","{'ALLY': {'orientation': 'NATURAL', 'aggregati...",allies,1105,313,27


In [17]:
run_query("""
CALL gds.wcc.stream('allies')
YIELD nodeId, componentId
WITH componentId, count(*) as members
WHERE members > 1
RETURN componentId, members
ORDER BY members DESC
LIMIT 5
""")

,componentId,members
0,0,195
1,26,4
2,245,3
3,6,2
4,2,2


The largest component of allies has 195 members. Then we have a couple of tiny allies islands with only a few members. If we visualize the largest component of allies in the Neo4j Browser and have the connect results nodes option selected, we get the following visualization.

Although we have found the largest allies component, we can observe that many of the characters in the component are actually enemies (red relationships). To better understand why this occurs, let’s look at the following example.

## Custom ally component algorithm
Suppose we wanted to find communities of allies where there are no enemies within the given component. The algorithm implementation is relatively straightforward, and you could use Neo4j custom procedures, for example. Still, if you are like me and don’t speak Java, you can always resort to your favorite scripting language. I have developed the custom Ally component algorithm in Python. First, we define some helper functions for fetching allies and enemies of a single node.

In [20]:
def get_allies(node_id):
    data = session.run("""MATCH (c:Character)-[:ALLY]-(ally) 
                          WHERE c.id = $node_id 
                          RETURN collect(ally.id) as allies""",
                      {'node_id':node_id})
    return data.single()['allies']

def get_enemies(node_id):
    data = session.run("""MATCH (c:Character)-[:ENEMY]-(enemy) 
                          WHERE c.id = $node_id 
                          RETURN collect(enemy.id) as allies""",
                      {'node_id':node_id})
    return data.single()['allies']

def get_members():
    return session.run("""
    CALL gds.wcc.stream('allies')
    YIELD nodeId, componentId
    WITH componentId, collect(gds.util.asNode(nodeId).id) as members
    WHERE size(members) > 10
    RETURN componentId, members
    """).single()['members']

My implementation is relatively simple. The input to the algorithm is the list of all node ids in the largest allied components. Start from a single node, load its enemies into the enemies list and load its allies into a queue that will be processed later. Then we iterate over the allied queue. If a node is not an enemy with any of the existing nodes in the component, add them to the community list and add their enemies to the community’s enemies list. I’ve added some minor performance tweaks like if we have traversed the node already in the allies queue, we can remove that node from the global list of starting nodes.

In [21]:
from collections import deque

def get_largest_stable_allies(node_list):
    final_communities = list()
    while node_list:
        community = set()
        enemies_list = set()
        visited = set()
        
        allies_list = deque()
        allies_list.appendleft(node_list[0])
        
        while allies_list:
            # Get the node from the queue
            start_node = allies_list.pop()
            
            # Skip if current node is enemy with anyone
            if start_node in enemies_list:
                continue
            
            # Get allies and enemies
            allies = get_allies(start_node)
            enemies = get_enemies(start_node)
            
            visited.add(start_node)
            # Add enemies
            enemies_list.update(enemies)
            # Add allies to the list of next visits
            allies_list.extendleft([x for x in allies if (x not in enemies_list) and (x not in visited)])
            # Add current node to community
            community.add(start_node)
            
            # Remove visited nodes from global node list
            try:
                node_list.remove(start_node)
            except:
                pass
        final_communities.append(list(community))
    return max(final_communities, key=len)

In [22]:
members = get_members()
get_largest_stable_allies(members)

['2253bd88-3f20-42f7-8540-cbf3674532da',
 '84888a4c-38a6-4c15-9ae5-e807934bf0a1',
 'c6c96262-9695-4e29-9e39-8870b5c6d373',
 'b869c676-a78c-4cfa-af42-bc17af1ef741',
 '7b8635c0-9a4a-4a10-a66d-43c79b4e505b',
 '9153ead5-bfb9-476c-95b1-db110bbdb336',
 '3c65ff51-d217-4f91-8cd5-4988e7709ced',
 'bd1d9edc-c59d-4ac2-9175-8f2e37888c7f',
 '69450170-f0e0-4d86-8484-268536b32925',
 '2e73d163-0c6e-4d14-815d-c42ca60b5c9e',
 'db7ff8c3-ba60-4218-8a44-50ce1c61fe78',
 'e6ba329c-99cd-433c-9a4b-7704542c7638',
 'bb2bd40e-f547-46b7-ab65-2f7f192bfd31',
 '51b6a301-e57c-48fb-950b-f37e3f19ecda',
 '952db63e-9d8f-480d-b77f-3b92a759d0ea',
 '6e75668b-dc28-479e-b6a5-01b6327fbd57',
 '1658c37f-3f02-4feb-8c18-ecd825ebbd85',
 '263a1e51-326b-45f7-91b4-298acbaa6291',
 '3ec99504-71f5-4300-bf94-e25748cadbdb',
 'cbe82e9f-ab46-4351-b3ed-b83bb9474c45',
 '2a19068f-cabc-4922-8004-1f1cb2c945a1',
 '1b838aa2-c05e-4a85-aaed-b5ffda25cdb5',
 'a4532c76-9c68-412e-a31c-6d51483354be',
 '69157309-1072-486c-a00d-4852989fa19f',
 '0f3d523c-204c-

In this code, the algorithm only returns the ids of nodes that belong to the largest allied component where there are no enemies within. It shouldn’t be a problem to mark these nodes in Neo4j, as you can match them by their ids. The largest component of allies, where there are no enemies within, has 142 members. If we visualize it in Neo4j Browser, we can see that there are no enemy relationships visible.

In [23]:
run_query("""
CALL gds.graph.drop('allies')
""")

,graphName,database,memoryUsage,sizeInBytes,nodeCount,relationshipCount,configuration,density,creationTime,modificationTime,schema
0,allies,neo4j,,-1,1105,313,{'relationshipProjection': {'ALLY': {'orientat...,0.000257,2022-04-17T13:30:32.452544000+00:00,2022-04-17T13:30:32.479791000+00:00,"{'relationships': {'ALLY': {}}, 'nodes': {'Cha..."


## Analyzing characters’ stats
In the last part of our analysis, we will examine the stats of the characters. We have the stats available for a total of 470 heroes. This information was scraped from Marvel’s website. The scale for stats ranges from zero to seven, and Iron Man does not have a single seven. Probably not the strongest of the heroes, even though he is one of the more popular ones. Now we will explore the characters with the highest stats average. Whenever I need some help with my cypher queries, I turn to Neo4j Slack. Luckily for us, Andrew Bowman is always around with great advice on optimizing and prettifying our cypher queries. This time he showed me the `apoc.map.values` procedure. It can be used to fetch all properties of a single node without explicitly writing the property keys.

In [24]:
run_query("""
MATCH (c:Character)-[:HAS_STATS]->(stats)
RETURN c.name as character, 
       apoc.coll.avg(apoc.map.values(stats, keys(stats))) as average_stats
ORDER BY average_stats DESC
LIMIT 10
""")

,character,average_stats
0,Sasquatch (Walter Langkowski),7.0
1,Squirrel Girl,7.0
2,Galactus,7.0
3,Deathstrike (Ultimate),7.0
4,GRAYDON CREED,7.0
5,CHTHON,7.0
6,UNREVEALED; GAEA IS HER GREEK NAME GAEA,7.0
7,SET,7.0
8,Rogue (X-Men: Battle of the Atom),7.0
9,Legion,7.0


It seems many characters have their stats maxed out. I am not sure exactly how this data collection process works, but I found a fascinating heroine by the name of Squirrel Girl that could probably kick Iron Man’s ass with one hand while making sourdough bread with the other. Or polish her nails, not exactly sure what type of girl she is. The only thing certain is that she is a badass.
## k-Nearest Neighbours algorithm
The k-Nearest Neighbour is one of the more standard graph algorithms and was already implemented in the Graph Data Science library before in the form of Cosine, Euclidian, and Pearson similarity algorithms. Those were basic implementation where the algorithms compared a given vector for all node pairs in the network. Because comparing all node pairs does not scale well, another implementation of the kNN algorithm was added to the library. It is based on the Efficient k-nearest neighbor graph construction for generic similarity measures article. Instead of comparing every node pair, the algorithm selects possible neighbors based on the assumption that the neighbors-of-neighbors of a node are most likely already the nearest one. The algorithm scales quasi-linear with respect to the node count instead of being quadratic. The implementation uses the Cosine similarity to compare two vectors.
First, we need to create a vector (array of numbers) that will be compared between the pairs of heroes. We will use the characters’ stats as well as their ability to fly to populate the vector. Because all stats have the same range between zero and seven, there is no need for normalization. We only need to encode the flight feature to span between zero and seven as well. Those characters that can fly will have the value of flight feature seven, while those who can’t fly will have the value zero.

In [25]:
run_query("""
MATCH (c:Character)-[:HAS_STATS]->(s)
WITH c, [s.durability, s.energy, s.fighting_skills, 
         s.intelligence, s.speed, s.strength,
         CASE WHEN c.flight = 'true' THEN 7 ELSE 0 END] as stats_vector
SET c.stats_vector = stats_vector
""")

""


We will also tag the characters that have the stats vector with a second label. This way, we can easily filter heroes with a stats vector in our native projection of the named graph.

In [26]:
run_query("""
MATCH (c:Character)
WHERE exists (c.stats_vector)
SET c:CharacterStats
""")

""


Now that everything is ready, we can go ahead and load our named graph. We will project all nodes with the CharacterStats label and their stats_vector properties in a named graph. If you need a quick refresher or introduction to how the GDS library works, I would suggest taking the Introduction to Graph Algorithms course.

In [27]:
run_query("""
CALL gds.graph.project('marvel', 'CharacterStats',
  '*', {nodeProperties:'stats_vector'})
""")

,nodeProjection,relationshipProjection,graphName,nodeCount,relationshipCount,projectMillis
0,"{'CharacterStats': {'label': 'CharacterStats',...","{'__ALL__': {'orientation': 'NATURAL', 'aggreg...",marvel,470,515,348


Now, we can go ahead and infer the similarity network with the new kNN algorithm. We will use the mutate mode of the algorithm. The mutate mode stores the results back to the projected graph instead of the Neo4j stored graph. This way, we can use the kNN algorithm results as the input for the community detection algorithms later in the workflow. The kNN algorithm has some parameters we can use to fine-tune the results:
* topK: The number of neighbors to find for each node. The K-nearest neighbors are returned.
* sampleRate: Sample rate to limit the number of comparisons per node.
* deltaThreshold: Value as a percentage to determine when to stop early. If fewer updates than the configured value happen, the algorithm stops.
* randomJoins: Between every iteration, how many attempts are being made to connect new node neighbors based on random selection.

We will define the topK value of 15 and sampleRate of 0.8, and leave the other parameters at default values.

In [29]:
run_query("""
CALL gds.knn.mutate('marvel', {nodeProperties:'stats_vector', 
  sampleRate:0.8, topK:15, mutateProperty:'score', mutateRelationshipType:'SIMILAR'})
""")

,ranIterations,nodePairsConsidered,didConverge,preProcessingMillis,computeMillis,mutateMillis,postProcessingMillis,nodesCompared,relationshipsWritten,similarityDistribution,configuration
0,6,455147,True,0,1477,199,-1,470,7050,"{'p1': 0.40000057220458984, 'max': 1.000006675...","{'topK': 15, 'maxIterations': 100, 'randomJoin..."


## Louvain Modularity algorithm
The similarity network is inferred and stored in the named graph. We can examine the community structure of this new similarity network with the Louvain Modularity algorithm. As the similarity scores of relationships are available as their properties, we will use the weighted variant of the Louvain Modularity algorithm. Using the `relationshipWeightProperty` parameter, we let the algorithm know it should consider the relationships’ weight when calculating the network’s community structure. This time we will use the `write` mode of the algorithm to store the results back to the Neo4j stored graph.

In [30]:
run_query("""
CALL gds.louvain.write('marvel',
  {relationshipTypes:['SIMILAR'],  
   relationshipWeightProperty:'score', 
   writeProperty:'louvain'});
""")

,writeMillis,nodePropertiesWritten,modularity,modularities,ranLevels,communityCount,communityDistribution,postProcessingMillis,preProcessingMillis,computeMillis,configuration
0,285,470,0.6083,"[0.5742237273581017, 0.6082996409215549]",2,7,"{'p99': 119, 'min': 16, 'max': 119, 'mean': 67...",4,0,1496,"{'maxIterations': 10, 'writeConcurrency': 4, '..."


We can examine the community structure results with the following cypher query.

In [31]:
run_query("""
MATCH (c:Character)-[:HAS_STATS]->(stats)
RETURN c.louvain as community, count(*) as members, 
       avg(stats.fighting_skills) as fighting_skills,
       avg(stats.durability) as durability,
       avg(stats.energy) as energy,
       avg(stats.intelligence) as intelligence,
       avg(stats.speed) as speed,
       avg(stats.strength) as strength,
       avg(CASE WHEN c.flight = 'true' THEN 7.0 ELSE 0.0 END) as flight
""")

,community,members,fighting_skills,durability,energy,intelligence,speed,strength,flight
0,144,119,4.252101,4.705882,3.697479,3.831933,3.857143,4.344538,0.588235
1,50,64,2.593750,2.218750,1.562500,2.750000,1.906250,1.921875,0.218750
2,235,77,3.948052,3.142857,2.324675,2.948052,2.974026,2.961039,0.636364
3,259,92,3.445652,3.554348,2.532609,3.326087,2.630435,3.086957,0.760870
4,120,16,4.500000,3.500000,2.375000,3.312500,3.125000,4.125000,0.000000
5,27,46,5.826087,6.739130,6.760870,6.413043,6.782609,6.565217,2.130435
6,67,56,4.660714,6.160714,5.267857,4.267857,5.696429,5.500000,2.375000


It would make sense to add the standard deviation for each stat, but it wouldn’t be presentable for a blog post. The community with an id 27 has the most powerful members. The average for most stats is 6.5, which means that they are almost entirely maxed out. The average value of flight at 2 indicates that around 30% (2/7) of the members can fly. The largest community with 106 members has their stats averaged between 2 and 3, which would indicate that they might be support characters with lesser abilities. The characters with stronger abilities are usually the lead characters.

## Label Propagation algorithm
Label Propagation algorithm can also be used to determine the community structure of a network. We will apply it to the inferred similarity network and compare the results with the Louvain Modularity algorithm results.

In [32]:
run_query("""
CALL gds.labelPropagation.write('marvel',
  {relationshipTypes:['SIMILAR'],
   relationshipWeightProperty:'score', 
   writeProperty:'labelPropagation'})
""")

,writeMillis,nodePropertiesWritten,ranIterations,didConverge,communityCount,communityDistribution,postProcessingMillis,preProcessingMillis,computeMillis,configuration
0,45,470,10,False,13,"{'p99': 147, 'min': 3, 'max': 147, 'mean': 36....",4,0,693,"{'maxIterations': 10, 'writeConcurrency': 4, '..."


We investigate the results of the Label Propagation algorithm.


In [33]:
run_query("""
MATCH (c:Character)-[:HAS_STATS]->(stats)
RETURN c.labelPropagation as community, count(*) as members, 
       avg(stats.fighting_skills) as fighting_skills,
       avg(stats.durability) as durability,
       avg(stats.energy) as energy,
       avg(stats.intelligence) as intelligence,
       avg(stats.speed) as speed,
       avg(stats.strength) as strength,
       avg(CASE WHEN c.flight = 'true' THEN 7.0 ELSE 0.0 END) as flight
""")

,community,members,fighting_skills,durability,energy,intelligence,speed,strength,flight
0,218,147,4.020408,4.639456,3.510204,3.782313,3.571429,4.197279,0.809524
1,343,37,3.297297,2.270270,1.783784,2.810811,2.405405,2.027027,1.135135
2,118,65,3.969231,3.430769,2.476923,3.153846,3.230769,3.292308,0.861538
3,270,14,5.285714,5.357143,4.714286,4.785714,4.642857,4.857143,0.000000
4,589,30,3.566667,3.433333,2.600000,3.033333,2.900000,3.033333,0.233333
5,215,16,4.500000,3.500000,2.375000,3.312500,3.125000,4.125000,0.000000
6,675,7,2.000000,1.000000,1.857143,3.000000,0.285714,0.857143,0.000000
7,722,10,3.100000,2.300000,0.400000,3.200000,2.500000,1.900000,0.000000
8,216,38,2.842105,2.763158,1.921053,2.736842,2.026316,2.342105,0.368421
9,73,47,5.829787,6.744681,6.617021,6.276596,6.787234,6.574468,2.085106


We can notice that the Label Propagation algorithm found twice as many communities as the Louvain Modularity algorithm. Some of them are relatively tiny. For example, the community with an id 395 has only three members, and all their average stats are at 1.0 value. They are the heroes that go by the name of Maggott, Deathbird, and Slayback. Funky names. The most powerful community has an id of 137 and only 23 members. Remember, the most powerful community found by the Louvain Modularity algorithm had 46 members and a slightly lower value of average stats.

## Conclusion
I hope you have learned some tricks on performing network analysis in Neo4j with the help of APOC and GDS libraries. There are still many things we could do with this graph, so expect a new post shortly.